# L1 Language Models, the Chat Format and Tokens

Here it talks about the same thing than the previous course:

- Base LLM
- Instruction tuned LLM


## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [3]:
# pip instal openai

import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

#### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course. 

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs. 

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to produce the desired results.

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output 
    )
    return response.choices[0].message["content"]

**Note**: This and all other lab notebooks of this course use OpenAI library version `0.27.0`. 

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the get_completion function: 

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Prompt the model and get a completion

This is for the Instruction Tuned LLM

In [5]:
response = get_completion("What is the capital of France?")

In [6]:
print(response)

The capital of France is Paris.


## Tokens

The way the model breaks down the words.

In [7]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

pilpolol


"lollipop" in reverse should be "popillol"

## An LLM does not predict the next word, it actually predicts the next token.

The model breaks down the words/phrase into tokens of veariable length, as seen with the word lollipop.

l + oll + ipop

so in order to reverse the letters, you can use the trick below, that way it tokenises each letter.

* For English language input, 1 token is around 4 characters, or 3/4 of a word. *

#### Token Limits
- Limits vary between models. Different models have different limits on the number of tokens in the input `context` + output completion.
- gpt3.5-turbo ~4000 tokens



In [8]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [9]:
response

'p-o-p-i-l-l-o-l'

## Helper function (chat format)
Here's the helper function we'll use in this course.

This is to make it like a chat format, which sometimes the LLM likes better, or we are able to get data on that format easily.

In [10]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

the format is like below.

In [11]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy little carrot so bright,
In the garden, a joyful sight.
With leaves so green and root so firm,
It beams with joy, its orange glow affirm.


More examples, this time with tone.

In [12]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a joyful carrot named Charlie who loved dancing in the sunlight.


In [13]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, grew a happy carrot, orange and light.


Below we can find a function that tells us how many tokens with their breakdown.

In [14]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [15]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [16]:
print(response)

Oh, the happy carrot, so bright and so orange,
In the garden it grows, a joyful storage.
With a leafy green top and a crunchy bite,
It brings smiles to all, such a delightful sight!


In [17]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 44, 'total_tokens': 81}


##### Token limits NOTE

This might be important in case the prompt provided exceeds the token limit for the model in use.

Prompt > 4000
or the combination exceeds the total allow by the model.

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

### How to safeguard your API key

In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
import os
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.